In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install -U evaluate
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -q mlflow nlp 2>/dev/null

import numpy as np
import pandas as pd
import re
import evaluate
import accelerate
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from nlp import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 803.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/WELFake_Dataset.csv").drop(['Unnamed: 0'], axis=1)
print(df.shape)

df['title'] = df['title'] + '\n\n\n' + df['text']
df = df[['title', 'label']]
df["title"].fillna('Null', inplace=True)
# df = df[~df['title'].isnull()]
print(df.shape)
df.sample(5).T

Mounted at /content/drive
(72134, 3)
(72134, 2)


<ipython-input-3-ca2ef281cd03>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title"].fillna('Null', inplace=True)


,62372,65751,21408,66685,9089
title,Italy says supports U.S. strikes on Syria\n\n\...,Donald Trump: President Obama Is ‘Founder Of ...,HOW MUSLIM IT WORKERS FOR Democrats Sold US In...,EXCLUSIVE: Foreign Minister: Turkey ’Very Clea...,Key Time In Collective Human History\n\n\nLeav...
label,0,1,1,0,1


In [4]:
# split test and training dataset
x_train, x_test, y_train, y_test = train_test_split(df["title"], df["label"], test_size=0.2, shuffle=True, random_state=42)
print(x_train.values[0:3])

['Elon Musk’s Tesla Stock Up $2 Billion Since Joining Trump’s Team\n\n\nAlthough Tesla CEO Elon Musk shocked Silicon Valley by breaking ranks to become an official member of the White House Manufacturing Jobs Initiative, the value of his Tesla stock is up by over $2 billion since the election of Donald Trump as President. [After a dizzying week of executive orders covering Obamacare, trade and immigration, President Trump held an approval rating of 55 percent of likely voters according to the Rasmussen Reports daily Presidential Tracking Poll. That is only slightly below the 59 percent level of President Obama when he left office.  Voters welcomed President Trump’s decision to scrap the   Partnership (TPP)   deal and agree that the North American Free Trade Agreement (NAFTA) with Mexico and Canada needs to be reworked. Although supporters argue free trade makes products cheaper for U. S. consumers, Americans believe  —   by a whopping 73 percent to 16 percent margin  —    it is more im

In [5]:
# prepare dataset for training
df_train = pd.DataFrame(columns=["title", "label"])
df_test = pd.DataFrame(columns=["title", "label"])
df_train["title"] = x_train
df_train["label"] = y_train
df_test["title"] = x_test
df_test["label"] = y_test
print(df_train[0:3])

df_train = Dataset.from_pandas(df_train)
df_test = Dataset.from_pandas(df_test)
print(df_train)

                                                   title  label
60264  Elon Musk’s Tesla Stock Up $2 Billion Since Jo...      0
42050  Wharton Business School Backers Seek Distance ...      0
62289  Border Patrol Agents Arrest Smuggler After Rol...      0
Dataset(features: {'title': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None)}, num_rows: 57707)


In [6]:
model_checkpoint = "google/bert_uncased_L-2_H-128_A-2"
max_len = 512

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(examples):
    return tokenizer(examples["title"], max_length=max_len, padding="max_length", truncation=True)

df_train = df_train.map(preprocess_function, batched=True)
df_test = df_test.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [7]:
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, hidden_dropout_prob=0.1)
model.resize_token_embeddings(len(tokenizer)) # need to resize due to new tokens added

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30522, 128, padding_idx=0)

In [8]:
# metric_name = 'f1'
metric_name = 'accuracy'
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"./snapshots/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=0.0005,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    fp16=True
)

In [9]:
# Computation of evaluation metric
metric = evaluate.load(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    # for f1 metric
    # return metric.compute(predictions=predictions, references=labels, average="micro")
    # for accuracy metric
    return metric.compute(predictions=predictions, references=labels)


In [10]:
trainer = Trainer(
    model,
    args,
    train_dataset=df_train,
    eval_dataset=df_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
train_log = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.081800,0.048981,0.987315
2,0.033400,0.042354,0.990850


In [12]:
trainer.evaluate()

{'eval_loss': 0.0423535592854023,
 'eval_accuracy': 0.9908504886670826,
 'eval_runtime': 25.0893,
 'eval_samples_per_second': 575.025,
 'eval_steps_per_second': 71.903,
 'epoch': 2.0}

In [13]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")
results = classifier(df_test["title"], max_length=max_len, padding="max_length", truncation=True)
dfResults = pd.DataFrame.from_dict(results)
dfResults['label'] = dfResults['label'].str.replace('LABEL_','')
# for f1 metric
# metric = metric.compute(predictions=dfResults['label'].tolist(), references=df_test["label"], average='micro')
# for accuracy metric
metric = metric.compute(predictions=dfResults['label'].tolist(), references=df_test["label"])
print(metric)

{'accuracy': 0.9908504886670826}
